Importing necessary libraries

In [2]:
!pip install geopy

In [4]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=a49a45c08b7b69c042179ea2bc62f6402a7f7811a250d2573cd4cd06433e5f0e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [48]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Reading the dataframe we prepared in previous question from csv file into dataframe df

In [49]:
df = pd.read_csv(r'D:\coursera\IBM data science\10_Applied_Data_Science_Capstone\TorontoDataset.csv')
df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [50]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Getting geographical coordinates of Toronto

In [51]:
toronto_add='Toronto, Ontario' #CITY ADDRESS
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(toronto_add)
latitude=location.latitude
longitude=location.longitude
print(F"The coordinates of Toronto are {latitude},{longitude}")

The coordinates of Toronto are 43.6534817,-79.3839347


Creating a map of Toronto city with all the postal codes

In [52]:
Toronto_map=folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postalcode,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['PostalCode'],df['Borough'],df['Neighborhood']):
    label='Postal Code:{},Neighborhoods:{},Borough:{}'.format(postalcode,neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],radius=5,color='blue',popup=label,
                        fill=True,fill_color='#3186cc',fill_opacity=0.7,
                        parse_html=True).add_to(Toronto_map)

In [53]:
Toronto_map

Defining foursquare credentials

In [54]:
import requests# library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize# tranform JSON file into a pandas dataframe

In [55]:
CLIENT_ID = "AQ5RCDARXWRSHMADWW2EQBRRWLXOIGDHJAJ21T13GLI5ETQJ"
CLIENT_SECRET = "E0V0HGYPTBLJOG2ZBT0H2F4LGXJIDDSN02GRA4FV1T1UN1NS"
VERSION = "20180605"

Create a function to extract dataframe of top 100 venues within 500m radius of each of the postal codes in Toronto

In [56]:
limit=100
def getVenues(codes,names,latitudes,longitudes,radius=500):
    venues_list=[]
    for code,name,lat,lng in zip(codes,names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(code,name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],
                          v['venue']['location']['lng'],
                          v['venue']['categories'][0]['name'])for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['PostalCode','Neighborhood','Neighborhood Latitude',
                          'Neighborhood Longitude','Venue name','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

Toronto Venues dataframe

In [57]:
Toronto_Venues=getVenues(codes=df['PostalCode'],
                               names=df['Neighborhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude'],radius=500)

In [58]:
Toronto_Venues.head()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue name,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [59]:
Toronto_Venues.shape

(2125, 8)

In [60]:
Toronto_Venues.groupby('PostalCode').count()[['Venue name']]

,Venue name
PostalCode,
M1B,1
M1C,1
M1E,9
M1G,4
M1H,8
...,...
M9N,1
M9P,8
M9R,4


In [61]:
print(F"There are {len(Toronto_Venues['Venue Category'].unique())} unique venue categories")

There are 267 unique venue categories


Now we analyze each neighborhood by creating a dataframe with one hot encoding

In [62]:
Toronto_onehot=pd.get_dummies(Toronto_Venues['Venue Category'],prefix='',prefix_sep='')#one hot encoding
#Add Neighborhood and PostalCode column back to Toronto_onehot dataframe
Toronto_onehot['PostalCode']=Toronto_Venues['PostalCode']
columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[columns]
Toronto_onehot.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
Toronto_onehot.shape

(2125, 268)

In [64]:
X=Toronto_Venues['Neighborhood']
Toronto_onehot[-1]=X
Toronto_onehot=Toronto_onehot.rename({-1:'Neighborhoods'},axis=1)
Toronto_onehot.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighborhoods
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village


In [65]:
col=[Toronto_onehot.columns[0]]+[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[1:-1])
Toronto_onehot=Toronto_onehot[col]
Toronto_onehot.head()

,PostalCode,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,Victoria Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,Victoria Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
Toronto_grouped=Toronto_onehot.groupby(['PostalCode','Neighborhoods'],sort=False).mean().reset_index()
Toronto_grouped.head()

,PostalCode,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,M4A,Victoria Village,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,M5A,"Regent Park,Harbourfront",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.020833,0.0,0.000000,0.020833
3,M6A,"Lawrence Manor,Lawrence Heights",0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.076923,0.000000
4,M7A,Ontario Provincial Government,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.033333


In [67]:
num=5
for code,nbd in zip(Toronto_grouped['PostalCode'][0:10],Toronto_grouped['Neighborhoods'][0:10]):
    print('----'+code+':'+nbd+'----')
    temp=Toronto_grouped[Toronto_grouped['PostalCode']==code].T.reset_index()
    temp=temp.iloc[2:]
    temp.columns=['venue category','freq']
    temp['freq']=temp['freq'].astype(float)
    temp.sort_values(by=['freq'],ascending=False,inplace=True)
    temp=temp.round({'freq':2})
    print(temp.reset_index(drop=True).head(num))

----M3A:Parkwoods----
                    venue category  freq
0             Fast Food Restaurant  0.33
1                             Park  0.33
2                Food & Drink Shop  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00
----M4A:Victoria Village----
          venue category  freq
0           Intersection  0.25
1  Portuguese Restaurant  0.25
2           Hockey Arena  0.25
3            Coffee Shop  0.25
4            Men's Store  0.00
----M5A:Regent Park,Harbourfront----
  venue category  freq
0    Coffee Shop  0.17
1           Park  0.06
2         Bakery  0.06
3           Café  0.06
4            Pub  0.06
----M6A:Lawrence Manor,Lawrence Heights----
           venue category  freq
0          Clothing Store  0.23
1  Furniture / Home Store  0.15
2       Accessories Store  0.08
3   Vietnamese Restaurant  0.08
4             Coffee Shop  0.08
----M7A:Ontario Provincial Government----
     venue category  freq
0       Coffee Shop  0.20
1  Sushi Res

Define a function to sort values of the venue in descending order

In [68]:
def most_common_venues(row,num_of_values):
    row_sorted=row.iloc[2:].sort_values(ascending=False)
    return row_sorted.index.values[0:num_of_values]

Create a dataframe to contain top 10 venues of each postal code

Create the name of columns of such dataframe

In [69]:
num_of_values=10
indicators=['st','nd','rd']
columns=['PostalCode','Neighborhoods']
for ind in np.arange(num_of_values):
    try:
        columns.append('{}{} Most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))

Initializing a dataframe

In [70]:
Toronto_top=pd.DataFrame(columns=columns)

Filling in the dataframe

In [71]:
Toronto_top['PostalCode']=Toronto_grouped['PostalCode']
Toronto_top['Neighborhoods']=Toronto_grouped['Neighborhoods']

In [72]:
for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_top.iloc[ind][2:]=most_common_venues(Toronto_grouped.iloc[ind],num_of_values)
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store
1,M4A,Victoria Village,Intersection,Portuguese Restaurant,Hockey Arena,Coffee Shop,Men's Store,Mediterranean Restaurant,Medical Center,Metro Station,Mexican Restaurant,Museum
2,M5A,"Regent Park,Harbourfront",Coffee Shop,Park,Bakery,Café,Pub,Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center
3,M6A,"Lawrence Manor,Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Athletics & Sports,Event Space,Arts & Crafts Store,Boutique,Women's Store
4,M7A,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Burrito Place,Smoothie Shop,Café,Sandwich Place,College Auditorium,College Cafeteria


In [73]:
#The features needed to run  kmeans clustering is the frequency of venue category.
#since K Means does not consider categorical variables, we drop the PostalCode and Neighborhoods columns from the Toronto_grouped dataframe
#run K means clustering to cluster the similar postal codes/Neighborhoods
Toronto_grouped.drop(['PostalCode','Neighborhoods'],axis=1,inplace=True)
Toronto_grouped.head()

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.020833,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.020833,0.0,0.000000,0.020833
3,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.076923,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.033333


In [74]:
k=5 # number of clusters
k_means=KMeans(n_clusters=k,random_state=0)
k_means.fit(Toronto_grouped)

KMeans(n_clusters=5, random_state=0)

In [75]:
k_means.labels_[0:10]

array([1, 1, 1, 1, 1, 3, 1, 1, 1, 1])

In [76]:
# Now, we add the cluster labels column to the Toronto_top dataframe 
Toronto_top.insert(0,' Cluster labels',k_means.labels_)
Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,1,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store
1,1,M4A,Victoria Village,Intersection,Portuguese Restaurant,Hockey Arena,Coffee Shop,Men's Store,Mediterranean Restaurant,Medical Center,Metro Station,Mexican Restaurant,Museum
2,1,M5A,"Regent Park,Harbourfront",Coffee Shop,Park,Bakery,Café,Pub,Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center
3,1,M6A,"Lawrence Manor,Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Athletics & Sports,Event Space,Arts & Crafts Store,Boutique,Women's Store
4,1,M7A,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Burrito Place,Smoothie Shop,Café,Sandwich Place,College Auditorium,College Cafeteria


In [77]:
#Now we merge the dataframe Toronto_top with df to add the latitude and longitude of each postal code
Toronto_data=df
Toronto_top=Toronto_top.join(Toronto_data.set_index('PostalCode'),on=['PostalCode'])

In [78]:
Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Borough,Neighborhood,Latitude,Longitude
0,1,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,Victoria Village,Intersection,Portuguese Restaurant,Hockey Arena,Coffee Shop,Men's Store,Mediterranean Restaurant,Medical Center,Metro Station,Mexican Restaurant,Museum,North York,Victoria Village,43.725882,-79.315572
2,1,M5A,"Regent Park,Harbourfront",Coffee Shop,Park,Bakery,Café,Pub,Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,1,M6A,"Lawrence Manor,Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Athletics & Sports,Event Space,Arts & Crafts Store,Boutique,Women's Store,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,1,M7A,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Burrito Place,Smoothie Shop,Café,Sandwich Place,College Auditorium,College Cafeteria,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [79]:
#We drop the Neighborhood column as it is duplicated as Neighborhoods column
Toronto_top.drop(['Neighborhood'],axis=1,inplace=True)

In [80]:
Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Borough,Latitude,Longitude
0,1,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store,North York,43.753259,-79.329656
1,1,M4A,Victoria Village,Intersection,Portuguese Restaurant,Hockey Arena,Coffee Shop,Men's Store,Mediterranean Restaurant,Medical Center,Metro Station,Mexican Restaurant,Museum,North York,43.725882,-79.315572
2,1,M5A,"Regent Park,Harbourfront",Coffee Shop,Park,Bakery,Café,Pub,Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center,Downtown Toronto,43.654260,-79.360636
3,1,M6A,"Lawrence Manor,Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Athletics & Sports,Event Space,Arts & Crafts Store,Boutique,Women's Store,North York,43.718518,-79.464763
4,1,M7A,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Burrito Place,Smoothie Shop,Café,Sandwich Place,College Auditorium,College Cafeteria,Queen's Park,43.662301,-79.389494


In [81]:
map_clusters=folium.Map([latitude,longitude],zoom_start=10)#Creating the map

In [82]:
x=np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [83]:
for code,nbd,lat,lng,clus in zip(Toronto_top['PostalCode'],Toronto_top['Neighborhoods'],Toronto_top['Latitude'],Toronto_top['Longitude'],Toronto_top[' Cluster labels']):
    label=folium.Popup('Postal Code:{};Neighborhoods:{};Cluster:{}'.format(code,nbd,clus),parse_html=True)
    folium.CircleMarker([lat,lng],popup=label,color=rainbow[clus-1],
                       fill=True,
                       fill_color=rainbow[clus-1],
                       fill_opacity=0.6,
                       radius=5).add_to(map_clusters)

In [84]:
map_clusters

In [85]:
Toronto_top.loc[Toronto_top[' Cluster labels']==0,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,0,M9M,"Humberlea,Emery",Baseball Field,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Malay Restaurant
1,0,M8Y,"Old Mill South,King's Mill Park,Sunnylea,Humbe...",Breakfast Spot,Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant


In [86]:
Toronto_top.loc[Toronto_top[' Cluster labels']==1,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,1,M3A,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store
1,1,M4A,Victoria Village,Intersection,Portuguese Restaurant,Hockey Arena,Coffee Shop,Men's Store,Mediterranean Restaurant,Medical Center,Metro Station,Mexican Restaurant,Museum
2,1,M5A,"Regent Park,Harbourfront",Coffee Shop,Park,Bakery,Café,Pub,Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center
3,1,M6A,"Lawrence Manor,Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Athletics & Sports,Event Space,Arts & Crafts Store,Boutique,Women's Store
4,1,M7A,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Burrito Place,Smoothie Shop,Café,Sandwich Place,College Auditorium,College Cafeteria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1,M4X,"St. James Town,Cabbagetown",Coffee Shop,Pizza Place,Café,Bakery,Restaurant,Chinese Restaurant,Pub,Italian Restaurant,General Entertainment,Taiwanese Restaurant
85,1,M5X,"First Canadian Place,Underground city",Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Bakery,Salad Place
86,1,M4Y,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Hotel,Bubble Tea Shop,Café
87,1,M7Y,Enclave of M4L,Comic Shop,Restaurant,Pizza Place,Fast Food Restaurant,Farmers Market,Burrito Place,Skate Park,Butcher,Light Rail Station,Garden


In [87]:
Toronto_top.loc[Toronto_top[' Cluster labels']==2,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,2,M3K,Downsview,Airport,Park,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
1,2,M2M,"Willowdale,Newtonbrook",Park,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
2,2,M9N,York(Weston),Park,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
3,2,M2P,York Mills West,Park,Convenience Store,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
4,2,M4W,Rosedale,Park,Playground,Trail,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
5,2,M8X,"The Kingsway,Montgomery Road,Old Mill North",Park,River,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant


In [88]:
Toronto_top.loc[Toronto_top[' Cluster labels']==3,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,3,M1B,"Malvern,Rouge",Fast Food Restaurant,Motel,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
1,3,M5N,Roselawn,Fast Food Restaurant,Garden,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Market,Middle Eastern Restaurant


In [89]:
Toronto_top.loc[Toronto_top[' Cluster labels']==4,Toronto_top.columns[:-3]].reset_index(drop=True)

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,4,M1C,"Rouge Hill,Port Union,Highland Creek",Bar,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Malay Restaurant
